In [ ]:
import numpy as np
import glob
import tqdm
from PIL import Image
import cv2 as cv
import os
from sklearn.metrics import confusion_matrix,cohen_kappa_score
from skimage import io
from skimage import measure
from scipy import ndimage
from sklearn.metrics import f1_score

In [ ]:
def compute_f1(prediction, target):
    """
    :param prediction: 2d array, int,
            estimated targets as returned by a classifier
    :param target: 2d array, int,
            ground truth
    :return:
        f1: float
    """
    prediction.tolist(), target.tolist()
    img, target = np.array(prediction).flatten(), np.array(target).flatten()
    f1 = f1_score(y_true=target, y_pred=img)
    return  f1

In [ ]:
list_f1=[]
for i in range(8):
    list_f1.append(compute_f1(all_pic2[i],all_pic[i]))

In [ ]:
def mean_iou(input, target, classes = 2):
    """  compute the value of mean iou
    :param input:  2d array, int, prediction
    :param target: 2d array, int, ground truth
    :param classes: int, the number of class
    :return:
        miou: float, the value of miou
    """
    miou = 0
    for i in range(classes):
        intersection = np.logical_and(target == i, input == i)
        # print(intersection.any())
        union = np.logical_or(target == i, input == i)
        temp = np.sum(intersection) / np.sum(union)
        miou += temp
    return  miou/classes

In [ ]:
list_mean_iou=[]
for i in range(8):
    list_mean_iou.append(mean_iou(all_pic2[i],all_pic[i],classes=2))

In [ ]:
def iou(input, target, classes=1):
    
    intersection = np.logical_and(target == classes, input == classes)
    
    union = np.logical_or(target == classes, input == classes)
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [ ]:
list_iou=[]
for i in range(8):
    list_iou.append(iou(all_pic2[i],all_pic[i],classes=1))

In [ ]:
def compute_kappa(prediction, target):
    """
    :param prediction: 2d array, int,
            estimated targets as returned by a classifier
    :param target: 2d array, int,
            ground truth
    :return:
        kappa: float
    """
    prediction.tolist(), target.tolist()
    img, target = np.array(prediction).flatten(), np.array(target).flatten()
    kappa = cohen_kappa_score(target, img)
    return  kappa

In [ ]:
list_kappa = []
for i in range(8):
    list_kappa.append(compute_kappa(np.array(all_pic2[i],dtype=int),np.array(all_pic[i],dtype=int)))

In [ ]:
list_acc=[]

In [ ]:
for i in range(8):
    matrix = confusion_matrix(y_true=np.array(all_pic[i]).flatten(), y_pred=np.array(all_pic2[i]).flatten())
    acc = np.diag(matrix).sum() / matrix.sum()
    list_acc.append(acc)

In [ ]:
import numpy as np
__all__ = ['SegmentationMetric']
 
"""
confusionMetric  # 注意：此处横着代表预测值，竖着代表真实值，与之前介绍的相反
P\L     P    N
P      TP    FP
N      FN    TN
"""
class SegmentationMetric(object):
    def __init__(self, numClass):
        self.numClass = numClass
        self.confusionMatrix = np.zeros((self.numClass,)*2)
 
    def pixelAccuracy(self):
        # return all class overall pixel accuracy
        #  PA = acc = (TP + TN) / (TP + TN + FP + TN)
        acc = np.diag(self.confusionMatrix).sum() /  self.confusionMatrix.sum()
        return acc
 
    def classPixelAccuracy(self):
        # return each category pixel accuracy(A more accurate way to call it precision)
        # acc = (TP) / TP + FP
        classAcc = np.diag(self.confusionMatrix) / self.confusionMatrix.sum(axis=1)
        return classAcc # 返回的是一个列表值，如：[0.90, 0.80, 0.96]，表示类别1 2 3各类别的预测准确率
 
    def meanPixelAccuracy(self):
        classAcc = self.classPixelAccuracy()
        meanAcc = np.nanmean(classAcc) # np.nanmean 求平均值，nan表示遇到Nan类型，其值取为0
        return meanAcc # 返回单个值，如：np.nanmean([0.90, 0.80, 0.96, nan, nan]) = (0.90 + 0.80 + 0.96） / 3 =  0.89
 
    def meanIntersectionOverUnion(self):
        # Intersection = TP Union = TP + FP + FN
        # IoU = TP / (TP + FP + FN)
        intersection = np.diag(self.confusionMatrix) # 取对角元素的值，返回列表
        union = np.sum(self.confusionMatrix, axis=1) + np.sum(self.confusionMatrix, axis=0) - np.diag(self.confusionMatrix) # axis = 1表示混淆矩阵行的值，返回列表； axis = 0表示取混淆矩阵列的值，返回列表 
        IoU = intersection / union  # 返回列表，其值为各个类别的IoU
        mIoU = np.nanmean(IoU) # 求各类别IoU的平均
        return mIoU
 
    def genConfusionMatrix(self, imgPredict, imgLabel): # 同FCN中score.py的fast_hist()函数
        # remove classes from unlabeled pixels in gt image and predict
        mask = (imgLabel >= 0) & (imgLabel < self.numClass)
        label = self.numClass * imgLabel[mask] + imgPredict[mask]
        count = np.bincount(label, minlength=self.numClass**2)
        confusionMatrix = count.reshape(self.numClass, self.numClass)
        return confusionMatrix
 
    def Frequency_Weighted_Intersection_over_Union(self):
        # FWIOU =     [(TP+FN)/(TP+FP+TN+FN)] *[TP / (TP + FP + FN)]
        freq = np.sum(self.confusion_matrix, axis=1) / np.sum(self.confusion_matrix)
        iu = np.diag(self.confusion_matrix) / (
                np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                np.diag(self.confusion_matrix))
        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU
 
 
    def addBatch(self, imgPredict, imgLabel):
        assert imgPredict.shape == imgLabel.shape
        self.confusionMatrix += self.genConfusionMatrix(imgPredict, imgLabel)
 
    def reset(self):
        self.confusionMatrix = np.zeros((self.numClass, self.numClass))
 
 
if __name__ == '__main__':
    list_pa=[]
    list_cpa=[]
    list_mpa=[]
    for i in range(8):
        imgPredict = np.array(all_pic[i],dtype=int) # 可直接换成预测图片
        imgLabel = np.array(all_pic2[i],dtype=int) # 可直接换成标注图片
        metric = SegmentationMetric(2) # 3表示有3个分类，有几个分类就填几
        metric.addBatch(imgPredict, imgLabel)
        pa = metric.pixelAccuracy()
        cpa = metric.classPixelAccuracy()
        mpa = metric.meanPixelAccuracy()
        mIoU = metric.meanIntersectionOverUnion()
        list_pa.append(pa)
        list_cpa.append(cpa[0])
        list_mpa.append(mpa)

In [1]:
from osgeo impor
import os
import numpy as np
import cv2
'''
input:原图的tif文件和预测的单通道png图像
out:合成新的tif,带有坐标信息
'''
class GRID:
 
    # 读图像文件
    def read_img(self, filename):
        dataset = gdal.Open(filename)  # 打开文件
 
        im_width = dataset.RasterXSize  # 栅格矩阵的列数
        im_height = dataset.RasterYSize  # 栅格矩阵的行数
 
        im_geotrans = dataset.GetGeoTransform()  # 仿射矩阵
        im_proj = dataset.GetProjection()  # 地图投影信息
        im_data = dataset.ReadAsArray(0, 0, im_width, im_height)  # 将数据写成数组，对应栅格矩阵
 
        del dataset  # 关闭对象，文件dataset
        return im_proj, im_geotrans, im_data, im_width, im_height
 
    # 写文件，以写成tif为例
    def write_img(self, filename, im_proj, im_geotrans, im_data):
 
        # 判断栅格数据的数据类型
        if 'int8' in im_data.dtype.name:
            datatype = gdal.GDT_Byte
        elif 'int16' in im_data.dtype.name:
            datatype = gdal.GDT_UInt16
        else:
            datatype = gdal.GDT_Float32
 
        # 判读数组维数
        if len(im_data.shape) == 3:
            im_bands, im_height, im_width = im_data.shape
        else:
            im_bands, (im_height, im_width) = 1, im_data.shape
 
        # 创建文件
        driver = gdal.GetDriverByName("GTiff")  # 数据类型必须有，因为要计算需要多大内存空间
        dataset = driver.Create(filename, im_width, im_height, im_bands, datatype)
 
        dataset.SetGeoTransform(im_geotrans)  # 写入仿射变换参数
        dataset.SetProjection(im_proj)  # 写入投影
 
        if im_bands == 1:
            dataset.GetRasterBand(1).WriteArray(im_data)  # 写入数组数据
        else:
            for i in range(im_bands):
                dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
 
        del dataset
 
 
if __name__ == "__main__":
    os.chdir(r'./data')  # 切换路径到待处理图像所在文件夹
    run = GRID()
    # 第一步
    proj, geotrans, data1, row1, column1 = run.read_img('pic_2.tif')  # 读数据,获取tif图像的信息
 
    img_path = 'pic_2.png' # 读取png图像数据
    data2 = cv2.imread(img_path, -1)
 
    data = np.array((data2), dtype=data1.dtype)  #数据格式
 
    run.write_img('pic_2_tif.tif', proj, geotrans, data)  # 生成tif

ImportError: cannot import name 'gdal' from 'osgeo' (C:\Users\zhou\anaconda3\lib\site-packages\osgeo\__init__.py)

In [3]:
import numpy as np

In [5]:
import glob
from PIL import Image
for i in glob.glob(r'./visualize_Test1.png'):
    im = Image.open(i,"r")
    print(i.split(".")[0])
    im.save("test2.tif".format(i.split(".")[0]),quality=95)


In [9]:
from matplotlib import pyplot as plt

In [2]:
import cv2

In [15]:
import glob
from PIL import Image
for i in glob.glob(r'./visualize_Test1.png'):
    im = Image.open(i,"r")
    print(i.split(".")[0])
    im.save("1.tif".format(i.split(".")[0]),quality=95)

In [16]:
im1=Image.open('./1.tif')

In [17]:
im2=np.array(im1)/255

In [19]:
cv2.imwrite('C:\\Users\\zhou\\Desktop\\11.png',im2)

True